# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [2]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

ModuleNotFoundError: No module named 'tensorflow'

First we'll load the text file and convert it into integers for our network to use. Here I'm creating a couple dictionaries to convert the characters to and from integers. Encoding the characters as integers makes it easier to use as input in the network.

In [1]:
with open('lotr_book_1.txt', 'r') as f:
    text=f.read()
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

NameError: name 'np' is not defined

Let's check out the first 100 characters, make sure everything is peachy. According to the [American Book Review](http://americanbookreview.org/100bestlines.asp), this is the 6th best first line of a book ever.

In [3]:
text[:100]

'V*art One \n\nTHE FELLOWSHIP \nOF THE RING \n\nJ.R.R.ToIkien \n\n\n\nComplete Table of Contents \n\n\n\nForeword '

And we can see the characters encoded as integers.

In [4]:
encoded[:100]

array([46,  7, 50, 67, 69,  1, 39, 63, 54,  1,  0,  0, 44, 32, 29,  1, 30,
       29, 36, 36, 39, 47, 43, 32, 33, 40,  1,  0, 39, 30,  1, 44, 32, 29,
        1, 42, 33, 38, 31,  1,  0,  0, 34, 10, 42, 10, 42, 10, 44, 64, 33,
       60, 58, 54, 63,  1,  0,  0,  0,  0, 27, 64, 62, 65, 61, 54, 69, 54,
        1, 44, 50, 51, 61, 54,  1, 64, 55,  1, 27, 64, 63, 69, 54, 63, 69,
       68,  1,  0,  0,  0,  0, 30, 64, 67, 54, 72, 64, 67, 53,  1], dtype=int32)

Since the network is working with individual characters, it's similar to a classification problem in which we are trying to predict the next character from the previous text.  Here's how many 'classes' our network has to pick from.

In [5]:
len(vocab)

79

## Making training mini-batches

Here is where we'll make our mini-batches for training. Remember that we want our batches to be multiple sequences of some desired number of sequence steps. Considering a simple example, our batches would look like this:

<img src="assets/sequence_batching@1x.png" width=500px>


<br>
We have our text encoded as integers as one long array in `encoded`. Let's create a function that will give us an iterator for our batches. I like using [generator functions](https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/) to do this. Then we can pass `encoded` into this function and get our batch generator.

The first thing we need to do is discard some of the text so we only have completely full batches. Each batch contains $N \times M$ characters, where $N$ is the batch size (the number of sequences) and $M$ is the number of steps. Then, to get the number of batches we can make from some array `arr`, you divide the length of `arr` by the batch size. Once you know the number of batches and the batch size, you can get the total number of characters to keep.

After that, we need to split `arr` into $N$ sequences. You can do this using `arr.reshape(size)` where `size` is a tuple containing the dimensions sizes of the reshaped array. We know we want $N$ sequences (`n_seqs` below), let's make that the size of the first dimension. For the second dimension, you can use `-1` as a placeholder in the size, it'll fill up the array with the appropriate data for you. After this, you should have an array that is $N \times (M * K)$ where $K$ is the number of batches.

Now that we have this array, we can iterate through it to get our batches. The idea is each batch is a $N \times M$ window on the array. For each subsequent batch, the window moves over by `n_steps`. We also want to create both the input and target arrays. Remember that the targets are the inputs shifted over one character. You'll usually see the first input character used as the last target character, so something like this:
```python
y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
```
where `x` is the input batch and `y` is the target batch.

The way I like to do this window is use `range` to take steps of size `n_steps` from $0$ to `arr.shape[1]`, the total number of steps in each sequence. That way, the integers you get from `range` always point to the start of a batch, and each window is `n_steps` wide.

In [6]:
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size
       n_seqs x n_steps from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       n_seqs: Batch size, the number of sequences per batch
       n_steps: Number of sequence steps per batch
    '''
    # Get the number of characters per batch and number of batches we can make
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * characters_per_batch]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # The features
        x = arr[:, n:n+n_steps]
        # The targets, shifted by one
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

Now I'll make my data sets and we can check out what's going on here. Here I'm going to use a batch size of 10 and 50 sequence steps.

In [7]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)

In [8]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[46  7 50 67 69  1 39 63 54  1]
 [63 53  1 58 69  1 72 50 68  1]
 [71 54 63 58 63 56  1 72 50 68]
 [ 1 69 64 64  1 68 69 70 65 58]
 [61 54 50 71 54  1 69 57 54  1]
 [63 10  1 33  1 72 58 61 61  1]
 [72 50 60 54 53  8  1  0  0 50]
 [58 68 69 50 63 52 54  1 50 57]
 [54 67 10  1  4 26 70 69  1 57]
 [63  1  0 37 58 53 53 61 54  9]]

y
 [[ 7 50 67 69  1 39 63 54  1  0]
 [53  1 58 69  1 72 50 68  1 65]
 [54 63 58 63 56  1 72 50 68  1]
 [69 64 64  1 68 69 70 65 58 53]
 [54 50 71 54  1 69 57 54  1 43]
 [10  1 33  1 72 58 61 61  1 69]
 [50 60 54 53  8  1  0  0 50 63]
 [68 69 50 63 52 54  1 50 57 54]
 [67 10  1  4 26 70 69  1 57 50]
 [ 1  0 37 58 53 53 61 54  9 54]]


If you implemented `get_batches` correctly, the above output should look something like 
```
x
 [[55 63 69 22  6 76 45  5 16 35]
 [ 5 69  1  5 12 52  6  5 56 52]
 [48 29 12 61 35 35  8 64 76 78]
 [12  5 24 39 45 29 12 56  5 63]
 [ 5 29  6  5 29 78 28  5 78 29]
 [ 5 13  6  5 36 69 78 35 52 12]
 [63 76 12  5 18 52  1 76  5 58]
 [34  5 73 39  6  5 12 52 36  5]
 [ 6  5 29 78 12 79  6 61  5 59]
 [ 5 78 69 29 24  5  6 52  5 63]]

y
 [[63 69 22  6 76 45  5 16 35 35]
 [69  1  5 12 52  6  5 56 52 29]
 [29 12 61 35 35  8 64 76 78 28]
 [ 5 24 39 45 29 12 56  5 63 29]
 [29  6  5 29 78 28  5 78 29 45]
 [13  6  5 36 69 78 35 52 12 43]
 [76 12  5 18 52  1 76  5 58 52]
 [ 5 73 39  6  5 12 52 36  5 78]
 [ 5 29 78 12 79  6 61  5 59 63]
 [78 69 29 24  5  6 52  5 63 76]]
 ```
 although the exact numbers will be different. Check to make sure the data is shifted over one step for `y`.

## Building the model

Below is where you'll build the network. We'll break it up into parts so it's easier to reason about each bit. Then we can connect them up into the whole network.

<img src="assets/charRNN.png" width=500px>


### Inputs

First off we'll create our input placeholders. As usual we need placeholders for the training data and the targets. We'll also create a placeholder for dropout layers called `keep_prob`.

def build_inputs(batch_size, num_steps):
    ''' Define placeholders for inputs, targets, and dropout 
    
        Arguments
        ---------
        batch_size: Batch size, number of sequences per batch
        num_steps: Number of sequence steps in a batch
        
    '''
    # Declare placeholders we'll feed into the graph
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Keep probability placeholder for drop out layers
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

### LSTM Cell

Here we will create the LSTM cell we'll use in the hidden layer. We'll use this cell as a building block for the RNN. So we aren't actually defining the RNN here, just the type of cell we'll use in the hidden layer.

We first create a basic LSTM cell with

```python
lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
```

where `num_units` is the number of units in the hidden layers in the cell. Then we can add dropout by wrapping it with 

```python
tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
```
You pass in a cell and it will automatically add dropout to the inputs or outputs. Finally, we can stack up the LSTM cells into layers with [`tf.contrib.rnn.MultiRNNCell`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/rnn/MultiRNNCell). With this, you pass in a list of cells and it will send the output of one cell into the next cell. Previously with TensorFlow 1.0, you could do this

```python
tf.contrib.rnn.MultiRNNCell([cell]*num_layers)
```

This might look a little weird if you know Python well because this will create a list of the same `cell` object. However, TensorFlow 1.0 will create different weight matrices for all `cell` objects. But, starting with TensorFlow 1.1 you actually need to create new cell objects in the list. To get it to work in TensorFlow 1.1, it should look like

```python
def build_cell(num_units, keep_prob):
    lstm = tf.contrib.rnn.BasicLSTMCell(num_units)
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
    
    return drop
    
tf.contrib.rnn.MultiRNNCell([build_cell(num_units, keep_prob) for _ in range(num_layers)])
```

Even though this is actually multiple LSTM cells stacked on each other, you can treat the multiple layers as one cell.

We also need to create an initial cell state of all zeros. This can be done like so

```python
initial_state = cell.zero_state(batch_size, tf.float32)
```

Below, we implement the `build_lstm` function to create these LSTM cells and the initial state.

def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

### RNN Output

Here we'll create the output layer. We need to connect the output of the RNN cells to a full connected layer with a softmax output. The softmax output gives us a probability distribution we can use to predict the next character.

If our input has batch size $N$, number of steps $M$, and the hidden layer has $L$ hidden units, then the output is a 3D tensor with size $N \times M \times L$. The output of each LSTM cell has size $L$, we have $M$ of them, one for each sequence step, and we have $N$ sequences. So the total size is $N \times M \times L$.

We are using the same fully connected layer, the same weights, for each of the outputs. Then, to make things easier, we should reshape the outputs into a 2D tensor with shape $(M * N) \times L$. That is, one row for each sequence and step, where the values of each row are the output from the LSTM cells.

One we have the outputs reshaped, we can do the matrix multiplication with the weights. We need to wrap the weight and bias variables in a variable scope with `tf.variable_scope(scope_name)` because there are weights being created in the LSTM cells. TensorFlow will throw an error if the weights created here have the same names as the weights created in the LSTM cells, which they will be default. To avoid this, we wrap the variables in a variable scope so we can give them unique names.

def build_output(lstm_output, in_size, out_size):
    ''' Build a softmax layer, return the softmax output and logits.
    
        Arguments
        ---------
        
        x: Input tensor
        in_size: Size of the input tensor, for example, size of the LSTM cells
        out_size: Size of this softmax layer
    
    '''

    # Reshape output so it's a bunch of rows, one row for each step for each sequence.
    # That is, the shape should be batch_size*num_steps rows by lstm_size columns
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Connect the RNN outputs to a softmax layer
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Since output is a bunch of rows of RNN cell outputs, logits will be a bunch
    # of rows of logit outputs, one for each step and sequence
    logits = tf.matmul(x, softmax_w) + softmax_b
    
    # Use softmax to get the probabilities for predicted characters
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

### Training loss

Next up is the training loss. We get the logits and targets and calculate the softmax cross-entropy loss. First we need to one-hot encode the targets, we're getting them as encoded characters. Then, reshape the one-hot targets so it's a 2D tensor with size $(M*N) \times C$ where $C$ is the number of classes/characters we have. Remember that we reshaped the LSTM outputs and ran them through a fully connected layer with $C$ units. So our logits will also have size $(M*N) \times C$.

Then we run the logits and targets through `tf.nn.softmax_cross_entropy_with_logits` and find the mean to get the loss.

In [12]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
    # One-hot encode targets and reshape to match logits, one row per batch_size per step
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

### Optimizer

Here we build the optimizer. Normal RNNs have have issues gradients exploding and disappearing. LSTMs fix the disappearance problem, but the gradients can still grow without bound. To fix this, we can clip the gradients above some threshold. That is, if a gradient is larger than that threshold, we set it to the threshold. This will ensure the gradients never grow overly large. Then we use an AdamOptimizer for the learning step.

In [13]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

### Build the network

Now we can put all the pieces together and build a class for the network. To actually run data through the LSTM cells, we will use [`tf.nn.dynamic_rnn`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/nn/dynamic_rnn). This function will pass the hidden and cell states across LSTM cells appropriately for us. It returns the outputs for each LSTM cell at each step for each sequence in the mini-batch. It also gives us the final LSTM state. We want to save this state as `final_state` so we can pass it to the first LSTM cell in the the next mini-batch run. For `tf.nn.dynamic_rnn`, we pass in the cell and initial state we get from `build_lstm`, as well as our input sequences. Also, we need to one-hot encode the inputs before going into the RNN. 

In [14]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Loss and optimizer (with gradient clipping)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

## Hyperparameters

Here I'm defining the hyperparameters for the network. 

* `batch_size` - Number of sequences running through the network in one pass.
* `num_steps` - Number of characters in the sequence the network is trained on. Larger is better typically, the network will learn more long range dependencies. But it takes longer to train. 100 is typically a good number here.
* `lstm_size` - The number of units in the hidden layers.
* `num_layers` - Number of hidden LSTM layers to use
* `learning_rate` - Learning rate for training
* `keep_prob` - The dropout keep probability when training. If you're network is overfitting, try decreasing this.

Here's some good advice from Andrej Karpathy on training the network. I'm going to copy it in here for your benefit, but also link to [where it originally came from](https://github.com/karpathy/char-rnn#tips-and-tricks).

> ## Tips and Tricks

>### Monitoring Validation Loss vs. Training Loss
>If you're somewhat new to Machine Learning or Neural Networks it can take a bit of expertise to get good models. The most important quantity to keep track of is the difference between your training loss (printed during training) and the validation loss (printed once in a while when the RNN is run on the validation data (by default every 1000 iterations)). In particular:

> - If your training loss is much lower than validation loss then this means the network might be **overfitting**. Solutions to this are to decrease your network size, or to increase dropout. For example you could try dropout of 0.5 and so on.
> - If your training/validation loss are about equal then your model is **underfitting**. Increase the size of your model (either number of layers or the raw number of neurons per layer)

> ### Approximate number of parameters

> The two most important parameters that control the model are `lstm_size` and `num_layers`. I would advise that you always use `num_layers` of either 2/3. The `lstm_size` can be adjusted based on how much data you have. The two important quantities to keep track of here are:

> - The number of parameters in your model. This is printed when you start training.
> - The size of your dataset. 1MB file is approximately 1 million characters.

>These two should be about the same order of magnitude. It's a little tricky to tell. Here are some examples:

> - I have a 100MB dataset and I'm using the default parameter settings (which currently print 150K parameters). My data size is significantly larger (100 mil >> 0.15 mil), so I expect to heavily underfit. I am thinking I can comfortably afford to make `lstm_size` larger.
> - I have a 10MB dataset and running a 10 million parameter model. I'm slightly nervous and I'm carefully monitoring my validation loss. If it's larger than my training loss then I may want to try to increase dropout a bit and see if that helps the validation loss.

> ### Best models strategy

>The winning strategy to obtaining very good models (if you have the compute time) is to always err on making the network larger (as large as you're willing to wait for it to compute) and then try different dropout values (between 0,1). Whatever model has the best validation performance (the loss, written in the checkpoint filename, low is good) is the one you should use in the end.

>It is very common in deep learning to run many different models with many different hyperparameter settings, and in the end take whatever checkpoint gave the best validation performance.

>By the way, the size of your training and validation splits are also parameters. Make sure you have a decent amount of data in your validation set or otherwise the validation performance will be noisy and not very informative.


In [15]:
batch_size = 100        # Sequences per batch
num_steps = 100         # Number of sequence steps per batch
lstm_size = 512         # Size of hidden layers in LSTMs
num_layers = 2          # Number of LSTM layers
learning_rate = 0.001   # Learning rate
keep_prob = 0.5         # Dropout keep probability

## Time for training

This is typical training code, passing inputs and targets into the network, then running the optimizer. Here we also get back the final LSTM state for the mini-batch. Then, we pass that state back into the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I save a checkpoint.

Here I'm saving checkpoints with the format

`i{iteration number}_l{# hidden layer units}.ckpt`

In [16]:
epochs = 20
# Save every N iterations
save_every_n = 200

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(encoded, batch_size, num_steps):
            counter += 1
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            
            end = time.time()
            print('Epoch: {}/{}... '.format(e+1, epochs),
                  'Training Step: {}... '.format(counter),
                  'Training loss: {:.4f}... '.format(batch_loss),
                  '{:.4f} sec/batch'.format((end-start)))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))
    
    saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, lstm_size))

Epoch: 1/20...  Training Step: 1...  Training loss: 4.3654...  3.1310 sec/batch
Epoch: 1/20...  Training Step: 2...  Training loss: 4.2582...  3.0188 sec/batch
Epoch: 1/20...  Training Step: 3...  Training loss: 3.6997...  3.0714 sec/batch
Epoch: 1/20...  Training Step: 4...  Training loss: 3.5236...  3.0290 sec/batch
Epoch: 1/20...  Training Step: 5...  Training loss: 3.5444...  3.0595 sec/batch
Epoch: 1/20...  Training Step: 6...  Training loss: 3.4527...  3.0601 sec/batch
Epoch: 1/20...  Training Step: 7...  Training loss: 3.3698...  3.0501 sec/batch
Epoch: 1/20...  Training Step: 8...  Training loss: 3.3597...  3.0372 sec/batch
Epoch: 1/20...  Training Step: 9...  Training loss: 3.3071...  3.0925 sec/batch
Epoch: 1/20...  Training Step: 10...  Training loss: 3.2661...  3.1306 sec/batch
Epoch: 1/20...  Training Step: 11...  Training loss: 3.2559...  3.2476 sec/batch
Epoch: 1/20...  Training Step: 12...  Training loss: 3.2647...  3.2661 sec/batch
Epoch: 1/20...  Training Step: 13... 

Epoch: 2/20...  Training Step: 103...  Training loss: 2.7783...  3.5492 sec/batch
Epoch: 2/20...  Training Step: 104...  Training loss: 2.7806...  3.5397 sec/batch
Epoch: 2/20...  Training Step: 105...  Training loss: 2.7545...  3.5345 sec/batch
Epoch: 2/20...  Training Step: 106...  Training loss: 2.7783...  3.5052 sec/batch
Epoch: 2/20...  Training Step: 107...  Training loss: 2.7389...  3.5562 sec/batch
Epoch: 2/20...  Training Step: 108...  Training loss: 2.7642...  3.5007 sec/batch
Epoch: 3/20...  Training Step: 109...  Training loss: 2.8371...  3.5809 sec/batch
Epoch: 3/20...  Training Step: 110...  Training loss: 2.7651...  3.5030 sec/batch
Epoch: 3/20...  Training Step: 111...  Training loss: 2.7347...  3.5629 sec/batch
Epoch: 3/20...  Training Step: 112...  Training loss: 2.7374...  3.5142 sec/batch
Epoch: 3/20...  Training Step: 113...  Training loss: 2.7480...  3.6613 sec/batch
Epoch: 3/20...  Training Step: 114...  Training loss: 2.7219...  3.8469 sec/batch
Epoch: 3/20...  

Epoch: 4/20...  Training Step: 203...  Training loss: 2.2767...  4.3403 sec/batch
Epoch: 4/20...  Training Step: 204...  Training loss: 2.2941...  3.8865 sec/batch
Epoch: 4/20...  Training Step: 205...  Training loss: 2.2960...  4.3801 sec/batch
Epoch: 4/20...  Training Step: 206...  Training loss: 2.2785...  4.4814 sec/batch
Epoch: 4/20...  Training Step: 207...  Training loss: 2.2873...  3.9071 sec/batch
Epoch: 4/20...  Training Step: 208...  Training loss: 2.2998...  4.5441 sec/batch
Epoch: 4/20...  Training Step: 209...  Training loss: 2.2831...  4.2314 sec/batch
Epoch: 4/20...  Training Step: 210...  Training loss: 2.2746...  3.9790 sec/batch
Epoch: 4/20...  Training Step: 211...  Training loss: 2.2808...  3.8293 sec/batch
Epoch: 4/20...  Training Step: 212...  Training loss: 2.2919...  3.8862 sec/batch
Epoch: 4/20...  Training Step: 213...  Training loss: 2.2620...  4.0604 sec/batch
Epoch: 4/20...  Training Step: 214...  Training loss: 2.2935...  4.3466 sec/batch
Epoch: 4/20...  

Epoch: 6/20...  Training Step: 303...  Training loss: 2.0913...  3.9057 sec/batch
Epoch: 6/20...  Training Step: 304...  Training loss: 2.0848...  3.8986 sec/batch
Epoch: 6/20...  Training Step: 305...  Training loss: 2.1017...  4.0087 sec/batch
Epoch: 6/20...  Training Step: 306...  Training loss: 2.1058...  3.7723 sec/batch
Epoch: 6/20...  Training Step: 307...  Training loss: 2.0842...  3.8101 sec/batch
Epoch: 6/20...  Training Step: 308...  Training loss: 2.0766...  3.6565 sec/batch
Epoch: 6/20...  Training Step: 309...  Training loss: 2.0973...  3.9203 sec/batch
Epoch: 6/20...  Training Step: 310...  Training loss: 2.0925...  4.4360 sec/batch
Epoch: 6/20...  Training Step: 311...  Training loss: 2.0598...  4.6905 sec/batch
Epoch: 6/20...  Training Step: 312...  Training loss: 2.0790...  4.5571 sec/batch
Epoch: 6/20...  Training Step: 313...  Training loss: 2.0819...  4.1568 sec/batch
Epoch: 6/20...  Training Step: 314...  Training loss: 2.0716...  3.6594 sec/batch
Epoch: 6/20...  

Epoch: 8/20...  Training Step: 403...  Training loss: 1.9544...  5.0029 sec/batch
Epoch: 8/20...  Training Step: 404...  Training loss: 1.9634...  3.9287 sec/batch
Epoch: 8/20...  Training Step: 405...  Training loss: 1.9630...  3.8334 sec/batch
Epoch: 8/20...  Training Step: 406...  Training loss: 1.9747...  3.6882 sec/batch
Epoch: 8/20...  Training Step: 407...  Training loss: 1.9556...  3.6524 sec/batch
Epoch: 8/20...  Training Step: 408...  Training loss: 1.9664...  3.7262 sec/batch
Epoch: 8/20...  Training Step: 409...  Training loss: 1.9570...  3.8497 sec/batch
Epoch: 8/20...  Training Step: 410...  Training loss: 1.9348...  4.1519 sec/batch
Epoch: 8/20...  Training Step: 411...  Training loss: 1.9364...  4.1892 sec/batch
Epoch: 8/20...  Training Step: 412...  Training loss: 1.9255...  4.4878 sec/batch
Epoch: 8/20...  Training Step: 413...  Training loss: 1.9323...  4.5655 sec/batch
Epoch: 8/20...  Training Step: 414...  Training loss: 1.9370...  4.0089 sec/batch
Epoch: 8/20...  

...

Epoch: 12/20...  Training Step: 602...  Training loss: 1.7630...  3.6855 sec/batch
Epoch: 12/20...  Training Step: 603...  Training loss: 1.7375...  3.7733 sec/batch
Epoch: 12/20...  Training Step: 604...  Training loss: 1.7549...  3.7173 sec/batch
Epoch: 12/20...  Training Step: 605...  Training loss: 1.7487...  3.7915 sec/batch
Epoch: 12/20...  Training Step: 606...  Training loss: 1.7624...  3.6169 sec/batch
Epoch: 12/20...  Training Step: 607...  Training loss: 1.7566...  3.7102 sec/batch
Epoch: 12/20...  Training Step: 608...  Training loss: 1.7479...  3.6561 sec/batch
Epoch: 12/20...  Training Step: 609...  Training loss: 1.7590...  3.6566 sec/batch
Epoch: 12/20...  Training Step: 610...  Training loss: 1.7524...  3.6913 sec/batch
Epoch: 12/20...  Training Step: 611...  Training loss: 1.7444...  3.7849 sec/batch
Epoch: 12/20...  Training Step: 612...  Training loss: 1.7593...  3.8028 sec/batch
Epoch: 12/20...  Training Step: 613...  Training loss: 1.7386...  3.7459 sec/batch
Epoc

Epoch: 13/20...  Training Step: 701...  Training loss: 1.6906...  3.5997 sec/batch
Epoch: 13/20...  Training Step: 702...  Training loss: 1.6919...  3.6775 sec/batch
Epoch: 14/20...  Training Step: 703...  Training loss: 1.7365...  3.7479 sec/batch
Epoch: 14/20...  Training Step: 704...  Training loss: 1.6776...  3.6784 sec/batch
Epoch: 14/20...  Training Step: 705...  Training loss: 1.6958...  3.6613 sec/batch
Epoch: 14/20...  Training Step: 706...  Training loss: 1.6734...  3.6463 sec/batch
Epoch: 14/20...  Training Step: 707...  Training loss: 1.6864...  3.5568 sec/batch
Epoch: 14/20...  Training Step: 708...  Training loss: 1.6807...  3.7015 sec/batch
Epoch: 14/20...  Training Step: 709...  Training loss: 1.6581...  3.6530 sec/batch
Epoch: 14/20...  Training Step: 710...  Training loss: 1.6660...  3.6100 sec/batch
Epoch: 14/20...  Training Step: 711...  Training loss: 1.6646...  3.6142 sec/batch
Epoch: 14/20...  Training Step: 712...  Training loss: 1.6810...  3.7038 sec/batch
Epoc

Epoch: 15/20...  Training Step: 800...  Training loss: 1.6183...  3.8355 sec/batch
Epoch: 15/20...  Training Step: 801...  Training loss: 1.6266...  3.5223 sec/batch
Epoch: 15/20...  Training Step: 802...  Training loss: 1.6190...  3.6450 sec/batch
Epoch: 15/20...  Training Step: 803...  Training loss: 1.6164...  3.7373 sec/batch
Epoch: 15/20...  Training Step: 804...  Training loss: 1.6139...  3.7753 sec/batch
Epoch: 15/20...  Training Step: 805...  Training loss: 1.6359...  3.9357 sec/batch
Epoch: 15/20...  Training Step: 806...  Training loss: 1.6032...  3.8279 sec/batch
Epoch: 15/20...  Training Step: 807...  Training loss: 1.5820...  3.6627 sec/batch
Epoch: 15/20...  Training Step: 808...  Training loss: 1.6389...  3.8007 sec/batch
Epoch: 15/20...  Training Step: 809...  Training loss: 1.6220...  3.7612 sec/batch
Epoch: 15/20...  Training Step: 810...  Training loss: 1.6202...  3.7558 sec/batch
Epoch: 16/20...  Training Step: 811...  Training loss: 1.6705...  3.5652 sec/batch
Epoc

Epoch: 17/20...  Training Step: 899...  Training loss: 1.5532...  4.2370 sec/batch
Epoch: 17/20...  Training Step: 900...  Training loss: 1.5700...  3.8270 sec/batch
Epoch: 17/20...  Training Step: 901...  Training loss: 1.5680...  3.7941 sec/batch
Epoch: 17/20...  Training Step: 902...  Training loss: 1.5597...  3.9698 sec/batch
Epoch: 17/20...  Training Step: 903...  Training loss: 1.5881...  4.2476 sec/batch
Epoch: 17/20...  Training Step: 904...  Training loss: 1.5654...  4.2765 sec/batch
Epoch: 17/20...  Training Step: 905...  Training loss: 1.5518...  4.3881 sec/batch
Epoch: 17/20...  Training Step: 906...  Training loss: 1.5789...  4.7457 sec/batch
Epoch: 17/20...  Training Step: 907...  Training loss: 1.5789...  4.6067 sec/batch
Epoch: 17/20...  Training Step: 908...  Training loss: 1.5696...  4.8358 sec/batch
Epoch: 17/20...  Training Step: 909...  Training loss: 1.5685...  4.8306 sec/batch
Epoch: 17/20...  Training Step: 910...  Training loss: 1.5755...  4.6464 sec/batch
Epoc

Epoch: 19/20...  Training Step: 998...  Training loss: 1.5426...  3.9963 sec/batch
Epoch: 19/20...  Training Step: 999...  Training loss: 1.5226...  3.8703 sec/batch
Epoch: 19/20...  Training Step: 1000...  Training loss: 1.5534...  3.8838 sec/batch
Epoch: 19/20...  Training Step: 1001...  Training loss: 1.5481...  3.7508 sec/batch
Epoch: 19/20...  Training Step: 1002...  Training loss: 1.5513...  4.0654 sec/batch
Epoch: 19/20...  Training Step: 1003...  Training loss: 1.5389...  4.0239 sec/batch
Epoch: 19/20...  Training Step: 1004...  Training loss: 1.5084...  4.0464 sec/batch
Epoch: 19/20...  Training Step: 1005...  Training loss: 1.5086...  4.2662 sec/batch
Epoch: 19/20...  Training Step: 1006...  Training loss: 1.5053...  4.5368 sec/batch
Epoch: 19/20...  Training Step: 1007...  Training loss: 1.5088...  4.2593 sec/batch
Epoch: 19/20...  Training Step: 1008...  Training loss: 1.5299...  3.6724 sec/batch
Epoch: 19/20...  Training Step: 1009...  Training loss: 1.5129...  3.6279 sec/

#### Saved checkpoints

Read up on saving and loading checkpoints here: https://www.tensorflow.org/programmers_guide/variables

In [17]:
tf.train.get_checkpoint_state('checkpoints')

model_checkpoint_path: "checkpoints/i1080_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i200_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i400_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i600_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i800_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1000_l512.ckpt"
all_model_checkpoint_paths: "checkpoints/i1080_l512.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [23]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [24]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

Here, pass in the path to a checkpoint and sample from the network.

In [25]:
tf.train.latest_checkpoint('checkpoints')

'checkpoints/i3960_l512.ckpt'

In [26]:
checkpoint = tf.train.latest_checkpoint('checkpoints')
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Farens. The man and this presence there was a long whell against the
serfice. And she would have seen a dest of sorry of any arrangements,
true. But the side house was in the money about the chief same, and
that he had to say to all his face, with some significance for his wife
that he had come and done to the penslition of a little and sounds of
the society, and always was in her and horses, and that that they
could not be a chalacteristic on her so second become in her soul that it
was all sincerity, and her clothest having been talking to horror of
a presine that was something was all his mother had no station.

The conversation had suddenly had not been anything.

Sometying her health she came over the services, had sat down again, and
at the conversation was not in the strange shreed to the door, and with
his sight of the party, the sick man thinking the secret of the mother
when they had brought a humored friends, and the prayers that he could not
let him any more and marking thi

In [27]:
checkpoint = 'checkpoints/i200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Fard,. ""
I"
"The tat he winse he wos al alt ong the hor an thot hed the
the sin the sand we he sos and se th the an soris he as oth thes sosed to he the he ad ond on har soron sha sas sitis he
sentes the he wis as arton thir ans on ond wome to whe the sos thes sosit to the sos at hos ho she sis the she him an tot thes th an setit and wo he angeras,. Tho hor ho wot thime
ne tot ha to we we he won thar al thir send tougtint ting and wh wamed, the the her what hot wit he wat ta hor,
thing th whe tousederede sad sithe the the he se the han shes oress ho tot on here an on ot to he the hhressasesid thise where ans andentor tat the
t the won he ans te hite wost hin sos torinnd
tare wes to se her
inntor ond tot ont ate
silt wens
thithe serede ho he the soros, sers on asis and af on he tare sin hos tot this won oth the he tins, and ang tee ar the ant oud thot woth shat ha lan and sithe to she sos thoth sar te sesilt ther singetin to he te artor to ho he
wans he ans her han ase and. 
I hind was

In [28]:
checkpoint = 'checkpoints/i600_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farnt out a seenen a doons the werly a prostines, bet of to as sone him, al her his and and she had booke word and wert on her had and said to hoult, sto stat to she whon he wond to
denting andond to stat had he he seed a sele to the corsions.

"The more when she shead, thourg that so he had and to the sort, sunding him his and and sace and where her
headed he said
the haress and than sheard her, her has thing his stalith his tongenter," she said
he dasted the monting the sond of
their that allong her ald
have a tare him that to the somesily wourd to dive in and som her and whene the sead, and withon te said whine and had sail he having had
same to sime her has
antanter that
she she distlidion of her heally.

"When, an the pinter on atone with
the pasting, and sooked hus a sant of her his her ander was having to the panting to shingher at a shiss to thow to and walk nover as to the portanion," said Stepan'Akknavitcch with which all she coundsess in this his,
boring the panitiou, and
at

In [29]:
checkpoint = 'checkpoints/i1200_l512.ckpt'
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farcial the
confiring to the mone of the correm and thinds. She
she saw the
streads of herself hand only astended of the carres to her his some of the princess of which he came him of
all that his white the dreasing of
thisking the princess and with she was she had
bettee a still and he was happined, with the pood on the mush to the peaters and seet it.

"The possess a streatich, the may were notine at his mate a misted
and the
man of the mother at the same of the seem her
felt. He had not here.

"I conest only be alw you thinking that the partion
of their said."

"A much then you make all her
somether. Hower their centing
about
this, and I won't give it in
himself.
I had not come at any see it will that there she chile no one that him.

"The distiction with you all.... It was
a mone of the mind were starding to the simple to a mone. It to be to ser in the place," said Vronsky.
"And a plais in
his face, has alled in the consess on at they to gan in the sint
at as that
he would not be a